# Experiment 1, Model 2

#### Model Setup

Run models in the following order, using their output labels as features for the next model:

[1.](#1) Multilabel Linguistic Classifier

[2.](#2) Multiclass Person Name + Occupation Sequence Classifier

[3.](#3) Multilabel Document Classifier

***

* Supervised learning
    * Train, Validate, and (Blind) Test Data: under directory `../data/token_clf_data/experiment_input/`
    * Prediction Data: Data: under directory `../data/token_clf_data/model_output/experiment1/`
* Word Embeddings
    * Custom fastText (word2vec with subwords) embeddings of 100 dimensions trained on the CRC Archives catalog's descriptive metadata (harvested October 2020)
    
***

**Table of Contents**

[I.](#i) Train the Person Name + Occupation Classifier

[II.](#ii) Predict Over All Data

Load programming resources:

In [1]:
# For custom functions and variables
import utils, utils1, config

# For data analysis
import pandas as pd
import numpy as np
import os, re

# For creating directories
from pathlib import Path

# For preprocessing
from gensim.models import FastText
from gensim import utils as gensim_utils

# For classification
import sklearn.metrics
from sklearn.preprocessing import MultiLabelBinarizer
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

Define resources for the models:

In [2]:
Path(config.experiment_input_path).mkdir(parents=True, exist_ok=True)    # For train, devtest, and blind test data
Path(config.experiment1_output_path).mkdir(parents=True, exist_ok=True)  # For predictions
Path(config.experiment1_agmt_path).mkdir(parents=True, exist_ok=True)    # For agreement metrics

In [3]:
# Model 1:
ling_label_subset = ["B-Generalization", "I-Generalization", "B-Gendered-Role", "I-Gendered-Role", "B-Gendered-Pronoun", "I-Gendered-Pronoun"]
# Model 2:
pers_o_label_subset = ["B-Unknown", "I-Unknown", "B-Feminine", "I-Feminine", "B-Masculine", "I-Masculine", "B-Occupation", "I-Occupation"]
# Model 3:
so_label_subset = ["B-Stereotype", "I-Stereotype", "B-Omission", "I-Omission"]

In [4]:
ling_label_tags = {
    "Gendered-Pronoun": ["B-Gendered-Pronoun", "I-Gendered-Pronoun"], "Gendered-Role": ["B-Gendered-Role", "I-Gendered-Role"],"Generalization": ["B-Generalization", "I-Generalization"]
    }
pers_o_label_tags = {
    "Unknown": ["B-Unknown", "I-Unknown"], "Feminine": ["B-Feminine", "I-Feminine"], "Masculine": ["B-Masculine", "I-Masculine"],
     "Occupation": ["B-Occupation", "I-Occupation"]
    }
so_label_tags = {
    "Stereotype": ["B-Stereotype", "I-Stereotype"], "Omission": ["B-Omission", "I-Omission"]
             }

In [5]:
d = 100                   # dimensions of word embeddings (should match utils1.py) for file names
target_labels = "pers_o"  # for file names

<a id="i"></a>
## I. Person Name + Occupation Labels

Train a multiclass sequence classifier, using Conditional Random Field with Adaptive Regularization of Weight Vectors (AROW), on the Person Name and Occupation labels, **passing in the Linguistic labels (not specific BIO label-tag pair) from the previous model's predictions as features to this model.**

Multiclass is a suitable setup for these labels because they are mutually exclusive (no one token should have more than one of these labels).  The sequence classifier with AROW was the highest performing for past algorithm experiments with sequence classifiers for Person Name and Occupation labels.

Load the Linguistic features:

In [6]:
a = "rf"
ling_filename = config.experiment1_agmt_path+"cc-{a}_ling_baseline_fastText{d}_evaluation_ALLDATA.csv".format(a=a,d=d)
ling_eval_df = pd.read_csv(ling_filename, usecols=["token_id", "predicted_tag"])
# Replace tags with labels
for label,tags in ling_label_tags.items():
    for tag in tags:
        ling_eval_df["predicted_tag"] = ling_eval_df["predicted_tag"].replace(to_replace=tag, value=label)
ling_features = ling_eval_df.rename(columns={"predicted_tag":"pred_ling_tag"})

In [7]:
train_df = pd.read_csv(config.tokc_path+"experiment_input/token_train.csv", index_col=0)
dev_df =  pd.read_csv(config.tokc_path+"experiment_input/token_validate.csv", index_col=0)
test_df = pd.read_csv(config.tokc_path+"experiment_input/token_test.csv", index_col=0)
perso_train = utils1.selectDataForLabels(train_df, "tag", pers_o_label_subset)
perso_dev = utils1.selectDataForLabels(dev_df, "tag", pers_o_label_subset)
perso_test = utils1.selectDataForLabels(test_df, "tag", pers_o_label_subset)
print(perso_train.shape, perso_dev.shape, perso_test.shape)

(308583, 10) (316721, 10) (153966, 10)


Join the linguistic labels (features) to the train, dev, and test data:

In [8]:
perso_train = perso_train.join(ling_features.set_index("token_id"), on="token_id", how="left")
assert perso_train.loc[perso_train.tag.isna()].shape[0] == 0
# perso_train.head()
perso_dev = perso_dev.join(ling_features.set_index("token_id"), on="token_id", how="left")
assert perso_dev.loc[perso_dev.tag.isna()].shape[0] == 0
perso_test = perso_test.join(ling_features.set_index("token_id"), on="token_id", how="left")
assert perso_test.loc[perso_test.tag.isna()].shape[0] == 0

In [9]:
perso_train.pred_ling_tag = perso_train.pred_ling_tag.fillna("O")
perso_train.pred_ling_tag.value_counts()

O                   306970
Gendered-Pronoun      1330
Gendered-Role          277
Generalization         116
Name: pred_ling_tag, dtype: int64

In [10]:
perso_dev.pred_ling_tag = perso_dev.pred_ling_tag.fillna("O")
perso_dev.pred_ling_tag.value_counts()

O                   315090
Gendered-Pronoun      1447
Gendered-Role          232
Generalization          83
Name: pred_ling_tag, dtype: int64

In [11]:
perso_test.pred_ling_tag = perso_test.pred_ling_tag.fillna("O")
perso_test.pred_ling_tag.value_counts()

O                   153132
Gendered-Pronoun       711
Gendered-Role          121
Generalization          56
Name: pred_ling_tag, dtype: int64

#### Preprocessing

In [12]:
train_df = perso_train.drop(columns=["description_id", "ann_id", "token_offsets", "field", "subset", "pos"])
dev_df = perso_dev.drop(columns=["description_id", "ann_id", "token_offsets", "field", "subset", "pos"])
test_df = perso_test.drop(columns=["description_id", "ann_id", "token_offsets", "field", "subset", "pos"])

In [13]:
df_train_token_groups = utils.implodeDataFrame(train_df, ['token_id', 'sentence_id', 'token'])
df_train_token_groups = df_train_token_groups.reset_index()
# df_train_token_groups.head()
df_train_grouped = utils.implodeDataFrame(df_train_token_groups, ['sentence_id'])
df_train_grouped = df_train_grouped.rename(columns={"token":"sentence"})
df_train_grouped = df_train_grouped.reset_index()

In [14]:
df_dev_token_groups = utils.implodeDataFrame(dev_df, ['token_id', 'sentence_id', 'token'])
df_dev_token_groups = df_dev_token_groups.reset_index()
df_dev_grouped = utils.implodeDataFrame(df_dev_token_groups, ['sentence_id'])
df_dev_grouped = df_dev_grouped.rename(columns={"token":"sentence"})
df_dev_grouped = df_dev_grouped.reset_index()
# df_dev_grouped.head()

In [15]:
df_test_token_groups = utils.implodeDataFrame(test_df, ['token_id', 'sentence_id', 'token'])
df_test_token_groups = df_test_token_groups.reset_index()
df_test_grouped = utils.implodeDataFrame(df_test_token_groups, ['sentence_id'])
df_test_grouped = df_test_grouped.rename(columns={"token":"sentence"})
df_test_grouped = df_test_grouped.reset_index()

Zip the linguistic label and BIO tags together with the tokens so each sentence item is a tuple: `(TOKEN, LING_LABEL, TAG_LIST)`

In [16]:
train_sentences_perso = utils1.zip2FeaturesAndTarget(df_train_grouped, "tag")
print(train_sentences_perso[2][:3])
dev_sentences_perso = utils1.zip2FeaturesAndTarget(df_dev_grouped, "tag")
print(dev_sentences_perso[0][:3])

[('Rev', ['O'], ['B-Masculine']), ('Tom', ['O'], ['I-Masculine']), ('Allan', ['O'], ['I-Masculine'])]
[('Title', ['O'], ['O']), (':', ['O'], ['O']), ('Papers', ['O'], ['O'])]


In [17]:
train_sentences = train_sentences_perso
dev_sentences = dev_sentences_perso

In [18]:
# Features
X_train = [utils1.extractSentenceFeatures(sentence) for sentence in train_sentences]
X_dev = [utils1.extractSentenceFeatures(sentence) for sentence in dev_sentences]
# Target
y_train = [utils1.extractSentenceTargets(sentence) for sentence in train_sentences]
y_dev = [utils1.extractSentenceTargets(sentence) for sentence in dev_sentences]

#### Train

Train a Conditional Random Field (CRF) model with the default parameters on the **Person Name** category of tags.  We'll increase the max iterations to 100 for this model.

In [19]:
a = "arow"
clf_perso = sklearn_crfsuite.CRF(algorithm=a, variance=0.5, max_iterations=100, all_possible_transitions=True)

In [20]:
# https://stackoverflow.com/questions/66059532/attributeerror-crf-object-has-no-attribute-keep-tempfiles
try:
    clf_perso.fit(X_train, y_train)
except AttributeError:
    pass

Remove `'O'` tags from the targets list since we are interested in the ability to apply the gendered and gender biased language related tags, and the `'O'` tags far outnumber the tags for gendered and gender biased language.

In [21]:
targets = list(clf_perso.classes_)
targets.remove('O')
print(targets)

['B-Masculine', 'I-Masculine', 'B-Occupation', 'I-Occupation', 'I-Unknown', 'B-Unknown', 'I-Feminine', 'B-Feminine']


#### Predict

In [22]:
predictions = clf_perso.predict(X_dev)

#### Evaluate: All Labels

In [23]:
print("  - F1:", metrics.flat_f1_score(y_dev, predictions, average="weighted", zero_division=0, labels=targets))
print("  - Prec:", metrics.flat_precision_score(y_dev, predictions, average="weighted", zero_division=0, labels=targets))
print("  - Rec", metrics.flat_recall_score(y_dev, predictions, average="weighted", zero_division=0, labels=targets))

  - F1: 0.42011630473949035
  - Prec: 0.4780338333125823
  - Rec 0.37724864263753516


Save the prediction data:

In [24]:
df_dev_grouped = df_dev_grouped.rename(columns={"tag":"tag_pers_o_expected"})
df_dev_grouped.insert(len(df_dev_grouped.columns), "tag_pers_o_predicted", predictions)
# df_dev_grouped.head()
df_dev_grouped = df_dev_grouped.set_index("sentence_id")
df_dev_exploded = df_dev_grouped.explode(list(df_dev_grouped.columns))
# df_dev_exploded.head()

In [25]:
filename = "crf_{a}_{t}_baseline_fastText{d}_predictions.csv".format(a=a, t=target_labels, d=d)
df_dev_exploded.to_csv(config.experiment1_output_path+filename)

#### Evaluate: Strict, Each Label

The built-in evaluation approach is strict, so unless the model predictions' labels are on text spans that exactly match the development data's test, the predicted labels will be deemed incorrect.

In [26]:
category = target_labels
filename = "crf_{a}_{c}_baseline_fastText{d}_predictions.csv".format(a=a, c=category, d=d)
pred_perso = pd.read_csv(config.experiment1_output_path+filename)
pred_perso = utils.getColumnValuesAsLists(pred_perso, "tag_{}_expected".format(category))
# pred_pers.head()

Calculate performance metrics for each category of labels:

In [27]:
pred_perso = pred_perso.fillna("O")
pred_perso = utils.isPredictedInExpected(pred_perso, "tag_{}_expected".format(category), "tag_{}_predicted".format(category), '_merge', 'O')
# pred_perso.head()

In [28]:
pred_perso_stats = utils.getScoresByCatTags(
    pred_perso, "_merge", pers_o_label_subset[0], "tag_{}_expected".format(category), "tag_{}_predicted".format(category), "token_id"
)
for i in range(1, len(pers_o_label_subset)):
    tag_stats = utils.getScoresByCatTags(
        pred_perso, "_merge", pers_o_label_subset[i], "tag_{}_expected".format(category), "tag_{}_predicted".format(category), "token_id"
    )
    pred_perso_stats = pd.concat([pred_perso_stats, tag_stats])
pred_perso_stats

,tag(s),false negative,false positive,true positive,precision,recall,f1
0,B-Unknown,1138,1027,1458,0.586720,0.561633,0.573903
0,I-Unknown,2316,1688,2448,0.591876,0.513854,0.550112
0,B-Feminine,46,82,134,0.620370,0.744444,0.676768
0,I-Feminine,177,427,584,0.577646,0.767411,0.659142
0,B-Masculine,358,412,469,0.532350,0.567110,0.549180
0,I-Masculine,844,620,695,0.528517,0.451592,0.487036
0,B-Occupation,539,313,578,0.648709,0.517457,0.575697
0,I-Occupation,761,456,634,0.581651,0.454480,0.510262


Save the statistics:

In [29]:
pred_perso_stats.to_csv(
    config.experiment1_agmt_path+"crf_{a}_baseline_fastText{d}_{c}_strict_agmt.csv".format(a=a, c=category, d=d)
)

#### Evaluate: Annotation Agreement

Calculate agreement at the annotation level, so if the model labels any word correctly from a manually annotated text span, that annotation is recorded as being correctly labeled (`true positive`).  Note whether the models' labels are an `exact_match`, `label_match`, `category_match` or `mismatch`.

Load the annotation data:

*Note: `ann_id` of `9999` indicates no annotation*

Group the annotation data by token:

In [41]:
df_ann = perso_dev[["sentence_id", "ann_id", "token_id", "tag"]]
df_ann = utils.implodeDataFrame(df_ann, ["sentence_id", "ann_id", "token_id"])
df_ann = df_ann.reset_index()
print(df_ann.shape)
df_ann.head()

(316566, 4)


,sentence_id,ann_id,token_id,tag
0,1,14384,7,[B-Unknown]
1,1,14384,8,[I-Unknown]
2,1,14384,9,[I-Unknown]
3,1,14384,10,[I-Unknown]
4,1,14384,11,[I-Unknown]


Align the columns of the dev and prediction DataFrames:

In [42]:
# Rename `sentence` column `token`
pred_perso = pred_perso.rename(columns={"sentence":"token"})
# pred_perso.head()

Join the data, adding the annotation IDs (`ann_id` column) to the prediction DataFrames:

In [43]:
index_list = ["sentence_id", "token_id"]

In [44]:
pred_perso_ann = pred_perso.join(df_ann.set_index(index_list), on=index_list, how="left")
pred_perso_ann = pred_perso_ann.drop(columns=["tag"])  # duplicate of tag_expected
assert pred_perso_ann.loc[pred_perso_ann["token_id"].isna()].shape[0] == 0
assert pred_perso_ann.loc[pred_perso_ann["ann_id"].isna()].shape[0] == 0
assert pred_perso_ann.loc[pred_perso_ann["tag_pers_o_predicted"].isna()].shape[0] == 0
assert pred_perso_ann.loc[pred_perso_ann["tag_pers_o_expected"].isna()].shape[0] == 0
# pred_perso_ann.head()

Explode the DataFrame:

In [45]:
pred_perso_ann = pred_perso_ann.explode(["tag_pers_o_expected"])

Generalize the BIO tags to label names:

In [46]:
# Get the predicted labels
pred_labels = list(pred_perso_ann["tag_{}_predicted".format(category)])
pred_labels = [label if label == "O" else label[2:] for label in pred_labels]
pred_perso_ann.insert(len(pred_perso_ann.columns), "label_{}_predicted".format(category), pred_labels)
# Get the lists of expected labels
exp_labels = list(pred_perso_ann["tag_{}_expected".format(category)])
exp_labels = [label if label == "O" else label[2:] for label in exp_labels]
pred_perso_ann.insert(len(pred_perso_ann.columns), "label_{}_expected".format(category), exp_labels)
# pred_perso_ann.head()

Group the data by annotation:

In [47]:
pred_perso_ann = pred_perso_ann.drop(columns=["tag_{}_expected".format(category), "tag_{}_predicted".format(category)])
pred_perso_ann = utils.implodeDataFrame(pred_perso_ann, ["sentence_id", "ann_id"])
pred_perso_ann = pred_perso_ann.reset_index()
# pred_perso_ann.head()

Record the agreements and disagreements:

In [48]:
agmt_types_perso, agmt_labels_perso = utils1.getAnnotationAgreement(pred_perso_ann, "label_pers_o_predicted", "label_pers_o_expected")
pred_perso_ann.insert(len(pred_perso_ann.columns), "annotation_agreement", agmt_types_perso)
pred_perso_ann.insert(len(pred_perso_ann.columns), "agreement_label", agmt_labels_perso)
# pred_perso_ann.head()

In [49]:
metrics_perso_all = utils1.getAnnotationAgreementMetrics(pred_perso_ann, "all")
metrics_perso_pn = utils1.getAnnotationAgreementMetrics(pred_perso_ann.loc[~(pred_perso_ann.agreement_label.isin(["Occupation","O"]))], "Person Name")
metrics_perso_unk = utils1.getAnnotationAgreementMetrics(pred_perso_ann.loc[pred_perso_ann.agreement_label == "Unknown"], "Unknown")
metrics_perso_fem = utils1.getAnnotationAgreementMetrics(pred_perso_ann.loc[pred_perso_ann.agreement_label == "Feminine"], "Feminine")
metrics_perso_mas = utils1.getAnnotationAgreementMetrics(pred_perso_ann.loc[pred_perso_ann.agreement_label == "Masculine"], "Masculine")
metrics_perso_occ = utils1.getAnnotationAgreementMetrics(pred_perso_ann.loc[pred_perso_ann.agreement_label == "Occupation"], "Occupation")
metrics_perso = pd.concat([metrics_perso_all, metrics_perso_pn, metrics_perso_unk, metrics_perso_fem, metrics_perso_mas, metrics_perso_occ])
metrics_perso

,labels,false negative,true positive,false positive,precision,recall,f_1
0,all,4951,5520,2243,0.711065,0.527170,0.605462
0,Person Name,4152,4805,1915,0.715030,0.536452,0.613000
0,Unknown,1878,3013,1133,0.726725,0.616029,0.666814
0,Feminine,401,595,264,0.692666,0.597390,0.641509
0,Masculine,1873,1197,518,0.697959,0.389902,0.500313
0,Occupation,799,715,328,0.685523,0.472259,0.559249


Save the metrics:

In [50]:
metrics_perso.to_csv(
    config.experiment1_agmt_path+"crf_{a}_baseline_fastText{d}_{c}_annot_agmt.csv".format(a=a, d=d, c=category)
)

#### Evaluate: Loose, Each Label

Generalize the tokens' BIO tags to the label, and calculate agreement scores for each label.

In [51]:
pred_perso_labels = pred_perso.drop(columns=["_merge"])
tag_exp = list(pred_perso_labels["tag_{}_expected".format(category)])
tag_pred = list(pred_perso_labels["tag_{}_predicted".format(category)])
label_exp = [[tag if tag == "O" else tag[2:] for tag in tag_exp_list] for tag_exp_list in tag_exp]
label_pred = [tag if tag == "O" else tag[2:] for tag in tag_pred]
pred_perso_labels = pred_perso_labels.drop(columns=["tag_{}_expected".format(category), "tag_{}_predicted".format(category)])
pred_perso_labels.insert(len(pred_perso_labels.columns), "label_{}_expected".format(category), label_exp)
pred_perso_labels.insert(len(pred_perso_labels.columns), "label_{}_predicted".format(category), label_pred)
# pred_pers_labels.loc[pred_pers_labels.label_personname_predicted == "Feminine"].head()  # Looks good

Calculate the agreement metrics at the label level for each token:

In [52]:
tags = ['Unknown', 'Feminine', 'Masculine', 'Occupation']
pred_perso_labels = utils.isPredictedInExpected(pred_perso_labels, "label_{}_expected".format(category), "label_{}_predicted".format(category), '_merge', 'O')

pred_perso_stats = utils.getScoresByCatTags(
    pred_perso_labels, "_merge", tags[0], "label_{}_expected".format(category), "label_{}_predicted".format(category), "token_id"
)
for i in range(1, len(tags)):
    tag_stats = utils.getScoresByCatTags(
        pred_perso_labels, "_merge", tags[i], "label_{}_expected".format(category), "label_{}_predicted".format(category), "token_id"
    )
    pred_perso_stats = pd.concat([pred_perso_stats, tag_stats])
pred_perso_stats

,tag(s),false negative,false positive,true positive,precision,recall,f1
0,Unknown,3441,2409,4212,0.636158,0.550372,0.590164
0,Feminine,221,378,849,0.691932,0.793458,0.739225
0,Masculine,1197,900,1296,0.590164,0.519856,0.552783
0,Occupation,1300,691,1290,0.651186,0.498069,0.564428


Combine and save the performance measures:

In [53]:
pred_perso_stats.to_csv(
    config.experiment1_agmt_path+"crf_{a}_baseline_fastText{d}_{c}_loose_agmt.csv".format(a=a, d=d, c=category)
)

<a id="ii"></a>
## II. Predict Over All Data

In [54]:
test_sentences = utils1.zip2FeaturesAndTarget(df_test_grouped, "tag")
X_test = [utils1.extractSentenceFeatures(sentence) for sentence in test_sentences]  # Features
y_test = [utils1.extractSentenceTargets(sentence) for sentence in test_sentences]   # Target
# Combine all data subsets' features and targets
X_all = X_train+X_dev+X_test
y_all = y_train+y_dev+y_test

In [55]:
all_predictions = clf_perso.predict(X_all)

#### Evaluate: All Labels

In [56]:
print("  - F1:", metrics.flat_f1_score(y_all, all_predictions, average="weighted", zero_division=0, labels=targets))
print("  - Prec:", metrics.flat_precision_score(y_all, all_predictions, average="weighted", zero_division=0, labels=targets))
print("  - Rec", metrics.flat_recall_score(y_all, all_predictions, average="weighted", zero_division=0, labels=targets))

  - F1: 0.5211899648086513
  - Prec: 0.5561442804328083
  - Rec 0.49419097345616575


Save the prediction data:

In [130]:
df_train_grouped = df_train_grouped.rename(columns={"tag":"tag_pers_o_expected"})
df_train_grouped = df_train_grouped.reset_index()
df_dev_grouped = df_dev_grouped.drop(columns="tag_{}_predicted".format(target_labels))
df_dev_grouped = df_dev_grouped.reset_index()
df_test_grouped = df_test_grouped.rename(columns={"tag":"tag_pers_o_expected"})
df_all_grouped = pd.concat([df_train_grouped, df_dev_grouped, df_test_grouped])
# df_all_grouped.head()

In [127]:
df_all_grouped.insert(len(df_all_grouped.columns), "tag_pers_o_predicted", all_predictions)
df_all_grouped = df_all_grouped.set_index("sentence_id")
df_all_exploded = df_all_grouped.explode(list(df_all_grouped.columns))
df_all_exploded.head()

,token_id,sentence,tag_pers_o_expected,pred_ling_tag,tag_pers_o_predicted
sentence_id,,,,,
2,16,Scope,[O],[O],O
2,17,and,[O],[O],O
2,18,Contents,[O],[O],O
2,19,:,[O],[O],O
2,20,Sermons,[O],[O],O


In [128]:
df_all_exploded = df_all_exploded.reset_index()

In [129]:
filename = "crf_{a}_{t}_baseline_fastText{d}_predictions_ALLDATA.csv".format(a=a, t=target_labels, d=d)
df_all_exploded.to_csv(config.experiment1_output_path+filename)

#### Evaluate: Each Label

The built-in evaluation approach is strict, so unless the model predictions' labels are on text spans that exactly match the manual annotations, the predicted labels will be deemed incorrect.

Calculate performance metrics for each category of labels:

In [131]:
pred_perso = df_all_exploded.copy()
pred_perso = pred_perso.fillna("O")
pred_perso = utils.isPredictedInExpected(pred_perso, "tag_{}_expected".format(category), "tag_{}_predicted".format(category), '_merge', 'O')
# pred_perso.head()

In [132]:
pred_perso_stats = utils.getScoresByCatTags(
    pred_perso, "_merge", pers_o_label_subset[0], "tag_{}_expected".format(category), "tag_{}_predicted".format(category), "token_id"
)
for i in range(1, len(pers_o_label_subset)):
    tag_stats = utils.getScoresByCatTags(
        pred_perso, "_merge", pers_o_label_subset[i], "tag_{}_expected".format(category), "tag_{}_predicted".format(category), "token_id"
    )
    pred_perso_stats = pd.concat([pred_perso_stats, tag_stats])
pred_perso_stats

,tag(s),false negative,false positive,true positive,precision,recall,f1
0,B-Unknown,2082,2433,4549,0.651533,0.686020,0.668332
0,I-Unknown,3959,3797,7738,0.670828,0.661537,0.666150
0,B-Feminine,88,183,392,0.681739,0.816667,0.743128
0,I-Feminine,337,1035,1620,0.610169,0.827798,0.702515
0,B-Masculine,678,875,1356,0.607799,0.666667,0.635873
0,I-Masculine,1491,1339,2103,0.610982,0.585142,0.597783
0,B-Occupation,896,623,1634,0.723970,0.645850,0.682682
0,I-Occupation,1304,892,1844,0.673977,0.585769,0.626785


Save the statistics:

In [133]:
pred_perso_stats.to_csv(
    config.experiment1_agmt_path+"crf_{a}_baseline_fastText{d}_{c}_strict_agmt_ALLDATA.csv".format(a=a, c=category, d=d)
)

#### Evaluate: Annotation Agreement

Calculate agreement at the annotation level, so if the model labels any word correctly from a manually annotated text span, that annotation is recorded as being correctly labeled (`true positive`).  Note whether the models' labels are an `exact_match`, `label_match`, `category_match` or `mismatch`.

Load the annotation data:

*Note: `ann_id` of `9999` indicates no annotation*

Group the annotation data by token:

In [134]:
perso_all = pd.concat([perso_train, perso_dev, perso_test])

In [135]:
df_ann = perso_all[["sentence_id", "ann_id", "token_id", "tag"]]
df_ann = utils.implodeDataFrame(df_ann, ["sentence_id", "ann_id", "token_id"])
df_ann = df_ann.reset_index()
print(df_ann.shape)
df_ann.head()

(778803, 4)


,sentence_id,ann_id,token_id,tag
0,0,99999,0,[O]
1,0,99999,1,[O]
2,0,99999,2,[O]
3,1,14384,7,[B-Unknown]
4,1,14384,8,[I-Unknown]


Align the columns of the annotation and prediction DataFrames:

In [138]:
# Rename `sentence` column `token`
pred_perso = pred_perso.rename(columns={"sentence":"token"}).sort_values(by="token_id")
pred_perso.head()

,sentence_id,token_id,token,tag_pers_o_expected,pred_ling_tag,tag_pers_o_predicted,_merge
604541,0,0,Identifier,[O],[O],O,true negative
604542,0,1,:,[O],[O],O,true negative
604543,0,2,AA5,[O],[O],O,true negative
298617,1,3,Title,[O],[O],O,true negative
298618,1,4,:,[O],[O],O,true negative


Join the data, adding the annotation IDs (`ann_id` column) to the prediction DataFrames:

In [139]:
index_list = ["sentence_id", "token_id"]

In [140]:
pred_perso_ann = pred_perso.join(df_ann.set_index(index_list), on=index_list, how="left")
pred_perso_ann = pred_perso_ann.drop(columns=["tag"])  # duplicate of tag_expected
assert pred_perso_ann.loc[pred_perso_ann["token_id"].isna()].shape[0] == 0
assert pred_perso_ann.loc[pred_perso_ann["ann_id"].isna()].shape[0] == 0
assert pred_perso_ann.loc[pred_perso_ann["tag_pers_o_predicted"].isna()].shape[0] == 0
assert pred_perso_ann.loc[pred_perso_ann["tag_pers_o_expected"].isna()].shape[0] == 0
# pred_perso_ann.head()

Explode the DataFrame:

In [141]:
pred_perso_ann = pred_perso_ann.explode(["tag_pers_o_expected"])

Generalize the BIO tags to label names:

In [142]:
# Get the predicted labels
pred_labels = list(pred_perso_ann["tag_{}_predicted".format(category)])
pred_labels = [label if label == "O" else label[2:] for label in pred_labels]
pred_perso_ann.insert(len(pred_perso_ann.columns), "label_{}_predicted".format(category), pred_labels)
# Get the lists of expected labels
exp_labels = list(pred_perso_ann["tag_{}_expected".format(category)])
exp_labels = [label if label == "O" else label[2:] for label in exp_labels]
pred_perso_ann.insert(len(pred_perso_ann.columns), "label_{}_expected".format(category), exp_labels)
# pred_perso_ann.head()

Group the data by annotation:

In [143]:
pred_perso_ann = pred_perso_ann.drop(columns=["tag_{}_expected".format(category), "tag_{}_predicted".format(category)])
pred_perso_ann = utils.implodeDataFrame(pred_perso_ann, ["sentence_id", "ann_id"])
pred_perso_ann = pred_perso_ann.reset_index()
# pred_perso_ann.head()

Record the agreements and disagreements:

In [144]:
agmt_types_perso, agmt_labels_perso = utils1.getAnnotationAgreement(pred_perso_ann, "label_pers_o_predicted", "label_pers_o_expected")
pred_perso_ann.insert(len(pred_perso_ann.columns), "annotation_agreement", agmt_types_perso)
pred_perso_ann.insert(len(pred_perso_ann.columns), "agreement_label", agmt_labels_perso)
# pred_perso_ann.head()

In [145]:
metrics_perso_all = utils1.getAnnotationAgreementMetrics(pred_perso_ann, "all")
metrics_perso_pn = utils1.getAnnotationAgreementMetrics(pred_perso_ann.loc[~(pred_perso_ann.agreement_label.isin(["Occupation","O"]))], "Person Name")
metrics_perso_unk = utils1.getAnnotationAgreementMetrics(pred_perso_ann.loc[pred_perso_ann.agreement_label == "Unknown"], "Unknown")
metrics_perso_fem = utils1.getAnnotationAgreementMetrics(pred_perso_ann.loc[pred_perso_ann.agreement_label == "Feminine"], "Feminine")
metrics_perso_mas = utils1.getAnnotationAgreementMetrics(pred_perso_ann.loc[pred_perso_ann.agreement_label == "Masculine"], "Masculine")
metrics_perso_occ = utils1.getAnnotationAgreementMetrics(pred_perso_ann.loc[pred_perso_ann.agreement_label == "Occupation"], "Occupation")
metrics_perso = pd.concat([metrics_perso_all, metrics_perso_pn, metrics_perso_unk, metrics_perso_fem, metrics_perso_mas, metrics_perso_occ])
metrics_perso

,labels,false negative,true positive,false positive,precision,recall,f_1
0,all,10080,15546,5003,0.756533,0.606649,0.673351
0,Person Name,8551,13603,4324,0.758800,0.614020,0.678775
0,Unknown,3749,8755,2649,0.767713,0.700176,0.732391
0,Feminine,817,1636,593,0.733961,0.666938,0.698847
0,Masculine,3985,3212,1082,0.748020,0.446297,0.559046
0,Occupation,1529,1943,679,0.741037,0.559620,0.637676


Save the metrics:

In [146]:
metrics_perso.to_csv(
    config.experiment1_agmt_path+"crf_{a}_baseline_fastText{d}_{c}_annot_agmt.csv".format(a=a, d=d, c=category)
)

#### Evaluation: Loose, Each Label

Generalize the tokens' BIO tags to the labels and calculate agreement scores for each label.

In [147]:
pred_perso_labels = pred_perso.drop(columns=["_merge"])
tag_exp = list(pred_perso_labels["tag_{}_expected".format(category)])
tag_pred = list(pred_perso_labels["tag_{}_predicted".format(category)])
label_exp = [[tag if tag == "O" else tag[2:] for tag in tag_exp_list] for tag_exp_list in tag_exp]
label_pred = [tag if tag == "O" else tag[2:] for tag in tag_pred]
pred_perso_labels = pred_perso_labels.drop(columns=["tag_{}_expected".format(category), "tag_{}_predicted".format(category)])
pred_perso_labels.insert(len(pred_perso_labels.columns), "label_{}_expected".format(category), label_exp)
pred_perso_labels.insert(len(pred_perso_labels.columns), "label_{}_predicted".format(category), label_pred)
# pred_pers_labels.loc[pred_pers_labels.label_personname_predicted == "Feminine"].head()  # Looks good

Calculate the agreement metrics at the label level for each token:

In [148]:
tags = ['Unknown', 'Feminine', 'Masculine', 'Occupation']
pred_perso_labels = utils.isPredictedInExpected(pred_perso_labels, "label_{}_expected".format(category), "label_{}_predicted".format(category), '_merge', 'O')

pred_perso_stats = utils.getScoresByCatTags(
    pred_perso_labels, "_merge", tags[0], "label_{}_expected".format(category), "label_{}_predicted".format(category), "token_id"
)
for i in range(1, len(tags)):
    tag_stats = utils.getScoresByCatTags(
        pred_perso_labels, "_merge", tags[i], "label_{}_expected".format(category), "label_{}_predicted".format(category), "token_id"
    )
    pred_perso_stats = pd.concat([pred_perso_stats, tag_stats])
pred_perso_stats

,tag(s),false negative,false positive,true positive,precision,recall,f1
0,Unknown,6017,5487,13030,0.703678,0.684097,0.693749
0,Feminine,422,874,2356,0.729412,0.848092,0.784288
0,Masculine,2159,1942,3731,0.657677,0.633447,0.645334
0,Occupation,2200,1381,3612,0.723413,0.621473,0.668579


Combine and save the performance measures:

In [149]:
pred_perso_stats.to_csv(
    config.experiment1_agmt_path+"crf_{a}_baseline_fastText{d}_{c}_loose_agmt.csv".format(a=a, d=d, c=category)
)